In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(hosts=[{'host': 'localhost', 'port': 9200}])

In [5]:
import reader, config, json
cfg = config.Config()

i = reader.read_investigation(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/i_Investigation.txt', 'r'))
result = {
    'id': 1,
    'sampleIds': [k for k in range(0, 17+1)],
    '_source': reader.conform_investigation_to_schema(
                reader.remove_isa_name_prefixes(
                  reader.remove_empty_values_in_dict(
                    reader.flatten_investigation(
                      i
                    )
                  )
                )
              )
}

In [7]:
es.index(index='cbit', doc_type='study', id=1, body=result['_source'])

{u'_id': u'1',
 u'_index': u'cbit',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'study',
 u'_version': 1,
 u'created': True}

In [13]:
i = reader.read_investigation(cfg, open('../../data/new_ISAcreatorArchives/StudyID_02_archive/i_Investigation.txt', 'r'))
result = {
  'id': 2,
  'sampleIds': [k for k in range(100, 153+1)],
  '_source': reader.conform_investigation_to_schema(
               reader.remove_isa_name_prefixes(
                 reader.remove_empty_values_in_dict(
                   reader.flatten_investigation(
                     i
                   )
                 )
               )
             )
}
es.index(index='cbit', doc_type='study', id=2, body=result['_source'])

{u'_id': u'2',
 u'_index': u'cbit',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'study',
 u'_version': 1,
 u'created': True}

In [19]:
import reader, config, json
cfg = config.Config()
a = reader.read_assay(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/a_transcription_micro_1.txt', 'r'))
s = reader.read_study_sample(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/s_study_sample.txt', 'r'))
d = reader.join_study_sample_and_assay(reader.clean_up_study_samples(s), reader.clean_up_assay(a))
d = reader.apply_special_treatments_to_study_sample(d)
a2 = reader.read_assay(cfg, open('../../data/new_ISAcreatorArchives/StudyID_02_archive/a_transcription_micro_1.txt', 'r'))
s2 = reader.read_study_sample(cfg, open('../../data/new_ISAcreatorArchives/StudyID_02_archive/s_study_sample.txt', 'r'))
d2 = reader.join_study_sample_and_assay(reader.clean_up_study_samples(s2), reader.clean_up_assay(a2))
d2 = reader.apply_special_treatments_to_study_sample(d2)

result = []

for i, (k, v) in enumerate(d.iteritems()):
    vv = v.copy()
    vv['Sample Name'] = k
    result.append({'id': i, 'studyId': 1, '_source': vv})

for i, (k, v) in enumerate(d2.iteritems()):
    vv = v.copy()
    vv['Sample Name'] = k
    result.append({'id': 100+i, 'studyId': 2, '_source': vv})

for r in result:
    print(r['id'])
    es.index(index='cbit', doc_type='sample', id=r['id'], body=r['_source'], refresh=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
100


RequestError: TransportError(400, u'mapper_parsing_exception', u'failed to parse [Source Name]')

In [21]:
result[17]

{'_source': {'*Compound': '8-br-cAMP - 8-bromo-cAMP',
  '*Material': 'PS - polystyrene',
  '*Strain': 'hMSC - human mesenchymal stromal cells',
  u'Array or chip design': u'Illumina HT12v3',
  u'Assay Type': u'in vitro',
  'Attach Duration (hours)': 12.5,
  u'Barcode': u'5342595028_C',
  u'Biological Replicate': 1,
  u'Biologically degradable': u'no',
  u'CAS number': u'23583-48-4',
  u'Cell type': u'stromal cell',
  u'Compound': u'8-bromo-cAMP',
  u'Compound abbreviation': u'8-br-cAMP',
  u'Control': False,
  u'Culture medium': u'alpha-MEM',
  'Dose (mM)': 1.0,
  'Dose Duration (hours)': 72.0,
  u'Gene expression type': u'Microarray',
  u'Group ID': 6,
  u'Material Class': u'polymer',
  u'Material Name': u'polystyrene',
  u'Material Shape': u'flat',
  u'Material abbreviation': u'PS',
  u'Organ': u'bone',
  u'Organism': u'Homo sapiens',
  u'Passage number': 2,
  u'Platform': u'Illumina',
  'Protocols': u'Protocol_01_Sample_treatment, Protocol_02_Material_synthesis',
  u'Sample ID': u'0

In [10]:
esResult = es.search(index='cbit', doc_type='study', body={})

In [12]:
print(json.dumps(esResult, indent=2, sort_keys=True))

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "hits": {
    "hits": [
      {
        "_id": "1", 
        "_index": "cbit", 
        "_score": 1.0, 
        "_source": {
          "INVESTIGATION": {
            "Investigation Identifier": "1478015927860", 
            "Investigation Title": "Investigation"
          }, 
          "ONTOLOGY SOURCE REFERENCE": {
            "Term Source Description": "Experimental Factor Ontology", 
            "Term Source File": "http://data.bioontology.org/ontologies/EFO", 
            "Term Source Name": "EFO", 
            "Term Source Version": "149"
          }, 
          "STUDY": {
            "Study Description": "Osteogenic differentiation of human mesenchymal stromal cells (hMSCs) may potentially be used in cell-based bone tissue-engineering applications to enhance the bone-forming potential of these cells. Osteogenic differentiation and adipogenic differentiation are thought to be mutually exclusive, and al

Taking a step back to follow the tutorial on [https://www.elastic.co/guide/en/elasticsearch/guide/current](`ElasticSearch: The Definitive Guide`)

In [23]:
es.index(index='megacorp', doc_type='employee', id=1, body={
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
})

{u'_id': u'1',
 u'_index': u'megacorp',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'employee',
 u'_version': 2,
 u'created': False}

In [24]:
es.index(index='megacorp', doc_type='employee', id=2, body={
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
})

{u'_id': u'2',
 u'_index': u'megacorp',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'employee',
 u'_version': 2,
 u'created': False}

In [25]:
es.index(index='megacorp', doc_type='employee', id=3, body={
    "first_name" :  "Douglas",
    "last_name" :   "Fir",
    "age" :         35,
    "about":        "I like to build cabinets",
    "interests":  [ "forestry" ]
})

{u'_id': u'3',
 u'_index': u'megacorp',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'employee',
 u'_version': 2,
 u'created': False}

In [26]:
es.get(index='megacorp', doc_type='employee', id=1)

{u'_id': u'1',
 u'_index': u'megacorp',
 u'_source': {u'about': u'I love to go rock climbing',
  u'age': 25,
  u'first_name': u'John',
  u'interests': [u'sports', u'music'],
  u'last_name': u'Smith'},
 u'_type': u'employee',
 u'_version': 2,
 u'found': True}

In [27]:
es.get(index='megacorp', doc_type='employee', id=2)

{u'_id': u'2',
 u'_index': u'megacorp',
 u'_source': {u'about': u'I like to collect rock albums',
  u'age': 32,
  u'first_name': u'Jane',
  u'interests': [u'music'],
  u'last_name': u'Smith'},
 u'_type': u'employee',
 u'_version': 2,
 u'found': True}

In [28]:
es.get(index='megacorp', doc_type='employee', id=3)

{u'_id': u'3',
 u'_index': u'megacorp',
 u'_source': {u'about': u'I like to build cabinets',
  u'age': 35,
  u'first_name': u'Douglas',
  u'interests': [u'forestry'],
  u'last_name': u'Fir'},
 u'_type': u'employee',
 u'_version': 2,
 u'found': True}

In [29]:
es.get(index='megacorp', doc_type='employee', id=4)

NotFoundError: TransportError(404, u'{"_index":"megacorp","_type":"employee","_id":"4","found":false}')

In [30]:
es.exists(index='megacorp', doc_type='employee', id=4)

False

In [31]:
es.exists(index='megacorp', doc_type='employee', id=3)

True

In [32]:
es.delete(index='megacorp', doc_type='employee', id=3)

{u'_id': u'3',
 u'_index': u'megacorp',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'employee',
 u'_version': 3,
 u'found': True}

In [33]:
es.exists(index='megacorp', doc_type='employee', id=3)

False

In [34]:
es.index(index='megacorp', doc_type='employee', id=3, body={
    "first_name" :  "Douglas",
    "last_name" :   "Fir",
    "age" :         35,
    "about":        "I like to build cabinets",
    "interests":  [ "forestry" ]
})

{u'_id': u'3',
 u'_index': u'megacorp',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'employee',
 u'_version': 4,
 u'created': True}

In [35]:
es.get(index='megacorp', doc_type='employee', id=3)

{u'_id': u'3',
 u'_index': u'megacorp',
 u'_source': {u'about': u'I like to build cabinets',
  u'age': 35,
  u'first_name': u'Douglas',
  u'interests': [u'forestry'],
  u'last_name': u'Fir'},
 u'_type': u'employee',
 u'_version': 4,
 u'found': True}

In [36]:
es.search(index='megacorp', doc_type='employee')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'2',
    u'_index': u'megacorp',
    u'_score': 1.0,
    u'_source': {u'about': u'I like to collect rock albums',
     u'age': 32,
     u'first_name': u'Jane',
     u'interests': [u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'},
   {u'_id': u'1',
    u'_index': u'megacorp',
    u'_score': 1.0,
    u'_source': {u'about': u'I love to go rock climbing',
     u'age': 25,
     u'first_name': u'John',
     u'interests': [u'sports', u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'},
   {u'_id': u'3',
    u'_index': u'megacorp',
    u'_score': 1.0,
    u'_source': {u'about': u'I like to build cabinets',
     u'age': 35,
     u'first_name': u'Douglas',
     u'interests': [u'forestry'],
     u'last_name': u'Fir'},
    u'_type': u'employee'}],
  u'max_score': 1.0,
  u'total': 3},
 u'timed_out': False,
 u'took': 18}

In [38]:
es.search(index='megacorp', doc_type='employee', q='last_name:Smith')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'2',
    u'_index': u'megacorp',
    u'_score': 0.30685282,
    u'_source': {u'about': u'I like to collect rock albums',
     u'age': 32,
     u'first_name': u'Jane',
     u'interests': [u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'},
   {u'_id': u'1',
    u'_index': u'megacorp',
    u'_score': 0.30685282,
    u'_source': {u'about': u'I love to go rock climbing',
     u'age': 25,
     u'first_name': u'John',
     u'interests': [u'sports', u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'}],
  u'max_score': 0.30685282,
  u'total': 2},
 u'timed_out': False,
 u'took': 3}

In [39]:
es.search(index='megacorp', doc_type='employee', body={
        "query": {
            "match": {
                "last_name": "Smith"
            }
        }
    })

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'2',
    u'_index': u'megacorp',
    u'_score': 0.30685282,
    u'_source': {u'about': u'I like to collect rock albums',
     u'age': 32,
     u'first_name': u'Jane',
     u'interests': [u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'},
   {u'_id': u'1',
    u'_index': u'megacorp',
    u'_score': 0.30685282,
    u'_source': {u'about': u'I love to go rock climbing',
     u'age': 25,
     u'first_name': u'John',
     u'interests': [u'sports', u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'}],
  u'max_score': 0.30685282,
  u'total': 2},
 u'timed_out': False,
 u'took': 6}

In [40]:
es.search(index='megacorp', doc_type='employee', body={
        "query": {
            "bool": {
                "must": {
                    "match": {
                        "last_name": "Smith"
                    }
                },
                "filter": {
                    "range": {
                        "age": { "gt": 30 }
                    }
                }
            }
        }
    })

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'2',
    u'_index': u'megacorp',
    u'_score': 0.30685282,
    u'_source': {u'about': u'I like to collect rock albums',
     u'age': 32,
     u'first_name': u'Jane',
     u'interests': [u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'}],
  u'max_score': 0.30685282,
  u'total': 1},
 u'timed_out': False,
 u'took': 9}

In [42]:
es.search(index='megacorp', doc_type='employee', body={
        "query": {
            "match": {
                "about": "rock climbing"
            }
        }
    })

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'megacorp',
    u'_score': 0.16273327,
    u'_source': {u'about': u'I love to go rock climbing',
     u'age': 25,
     u'first_name': u'John',
     u'interests': [u'sports', u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'},
   {u'_id': u'2',
    u'_index': u'megacorp',
    u'_score': 0.016878016,
    u'_source': {u'about': u'I like to collect rock albums',
     u'age': 32,
     u'first_name': u'Jane',
     u'interests': [u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'}],
  u'max_score': 0.16273327,
  u'total': 2},
 u'timed_out': False,
 u'took': 5}

In [43]:
es.search(index='megacorp', doc_type='employee', body={
        "query": {
            "match_phrase": {
                "about": "rock climbing"
            }
        }
    })

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'megacorp',
    u'_score': 0.23013961,
    u'_source': {u'about': u'I love to go rock climbing',
     u'age': 25,
     u'first_name': u'John',
     u'interests': [u'sports', u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee'}],
  u'max_score': 0.23013961,
  u'total': 1},
 u'timed_out': False,
 u'took': 14}

In [45]:
es.search(index='megacorp', doc_type='employee', body={
        "query": {
            "match_phrase": {
                "about": "rock climbing"
            }
        },
        "highlight": {
            "fields": {
                "about": {}
            }
        }
    })

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'megacorp',
    u'_score': 0.23013961,
    u'_source': {u'about': u'I love to go rock climbing',
     u'age': 25,
     u'first_name': u'John',
     u'interests': [u'sports', u'music'],
     u'last_name': u'Smith'},
    u'_type': u'employee',
    u'highlight': {u'about': [u'I love to go <em>rock</em> <em>climbing</em>']}}],
  u'max_score': 0.23013961,
  u'total': 1},
 u'timed_out': False,
 u'took': 35}

In [47]:
r = es.search(index='megacorp', doc_type='employee', body={
        "aggs": {
            "all_interests": {
                "terms": {
                    "field": "interests"
                }
            }
        }
    })
print(json.dumps(r, indent=2, sort_keys=True))

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "all_interests": {
      "buckets": [
        {
          "doc_count": 2, 
          "key": "music"
        }, 
        {
          "doc_count": 1, 
          "key": "forestry"
        }, 
        {
          "doc_count": 1, 
          "key": "sports"
        }
      ], 
      "doc_count_error_upper_bound": 0, 
      "sum_other_doc_count": 0
    }
  }, 
  "hits": {
    "hits": [
      {
        "_id": "2", 
        "_index": "megacorp", 
        "_score": 1.0, 
        "_source": {
          "about": "I like to collect rock albums", 
          "age": 32, 
          "first_name": "Jane", 
          "interests": [
            "music"
          ], 
          "last_name": "Smith"
        }, 
        "_type": "employee"
      }, 
      {
        "_id": "1", 
        "_index": "megacorp", 
        "_score": 1.0, 
        "_source": {
          "about": "I love to go rock climbing", 
        

In [48]:
r = es.search(index='megacorp', doc_type='employee', body={
        "query": {
            "match": {
                "last_name": "Smith"
            }
        },
        "aggs": {
            "all_interests": {
                "terms": {
                    "field": "interests"
                }
            }
        }
    })
print(json.dumps(r, indent=2, sort_keys=True))

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "all_interests": {
      "buckets": [
        {
          "doc_count": 2, 
          "key": "music"
        }, 
        {
          "doc_count": 1, 
          "key": "sports"
        }
      ], 
      "doc_count_error_upper_bound": 0, 
      "sum_other_doc_count": 0
    }
  }, 
  "hits": {
    "hits": [
      {
        "_id": "2", 
        "_index": "megacorp", 
        "_score": 0.30685282, 
        "_source": {
          "about": "I like to collect rock albums", 
          "age": 32, 
          "first_name": "Jane", 
          "interests": [
            "music"
          ], 
          "last_name": "Smith"
        }, 
        "_type": "employee"
      }, 
      {
        "_id": "1", 
        "_index": "megacorp", 
        "_score": 0.30685282, 
        "_source": {
          "about": "I love to go rock climbing", 
          "age": 25, 
          "first_name": "John", 
          "inter

In [51]:
r = es.search(index='megacorp', doc_type='employee', body={
        "aggs": {
            "all_interests": {
                "terms": {"field": "interests"},
                "aggs": {
                    "avg_age": {
                        "avg": {"field": "age"}
                    }
                }
            }
        }
    })
print(json.dumps(r, indent=2, sort_keys=True))

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "aggregations": {
    "all_interests": {
      "buckets": [
        {
          "avg_age": {
            "value": 28.5
          }, 
          "doc_count": 2, 
          "key": "music"
        }, 
        {
          "avg_age": {
            "value": 35.0
          }, 
          "doc_count": 1, 
          "key": "forestry"
        }, 
        {
          "avg_age": {
            "value": 25.0
          }, 
          "doc_count": 1, 
          "key": "sports"
        }
      ], 
      "doc_count_error_upper_bound": 0, 
      "sum_other_doc_count": 0
    }
  }, 
  "hits": {
    "hits": [
      {
        "_id": "2", 
        "_index": "megacorp", 
        "_score": 1.0, 
        "_source": {
          "about": "I like to collect rock albums", 
          "age": 32, 
          "first_name": "Jane", 
          "interests": [
            "music"
          ], 
          "last_name": "Smith"
        }, 
        "_ty

---

Back to cBiT

---

In [52]:
import reader, config, json
cfg = config.Config()

i = reader.read_investigation(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/i_Investigation.txt', 'r'))
result = {
    'id': 1,
    'sampleIds': [k for k in range(0, 17+1)],
    '_source': reader.conform_investigation_to_schema(
                reader.remove_isa_name_prefixes(
                  reader.remove_empty_values_in_dict(
                    reader.flatten_investigation(
                      i
                    )
                  )
                )
              )
}
es.index(index='cbit', doc_type='study', id=1, body=result['_source'])

{u'_id': u'1',
 u'_index': u'cbit',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'study',
 u'_version': 1,
 u'created': True}

In [53]:
# Only deal with Study 1 for the moment
import reader, config, json
cfg = config.Config()
a = reader.read_assay(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/a_transcription_micro_1.txt', 'r'))
s = reader.read_study_sample(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/s_study_sample.txt', 'r'))
d = reader.join_study_sample_and_assay(reader.clean_up_study_samples(s), reader.clean_up_assay(a))
d = reader.apply_special_treatments_to_study_sample(d)

result = []

for i, (k, v) in enumerate(d.iteritems()):
    vv = v.copy()
    vv['Sample Name'] = k
    result.append({'id': i, 'studyId': 1, '_source': vv})

for r in result:
    print(r['id'])
    es.index(index='cbit', doc_type='sample', id=r['id'], body=r['_source'], refresh=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [64]:
result[1]

{'_source': {'*Compound': 'db-cAMP - dibutyryl-cAMP',
  '*Material': 'PS - polystyrene',
  '*Strain': 'hMSC - human mesenchymal stromal cells',
  u'Array or chip design': u'Illumina HT12v3',
  u'Assay Type': u'in vitro',
  'Attach Duration (hours)': 12.5,
  u'Barcode': u'5342595019_E',
  u'Biological Replicate': 2,
  u'Biologically degradable': u'no',
  u'CAS number': u'362-74-3',
  u'Cell type': u'stromal cell',
  u'Compound': u'dibutyryl-cAMP',
  u'Compound abbreviation': u'db-cAMP',
  u'Control': False,
  u'Culture medium': u'alpha-MEM',
  'Dose (mM)': 1.0,
  'Dose Duration (hours)': 6.0,
  u'Gene expression type': u'Microarray',
  u'Group ID': 2,
  u'Material Class': u'polymer',
  u'Material Name': u'polystyrene',
  u'Material Shape': u'flat',
  u'Material abbreviation': u'PS',
  u'Organ': u'bone',
  u'Organism': u'Homo sapiens',
  u'Passage number': 2,
  u'Platform': u'Illumina',
  'Protocols': u'Protocol_01_Sample_treatment, Protocol_02_Material_synthesis',
  u'Sample ID': u'01-2

In [7]:
import json
def p(doc, max_lines=25):
    lines = json.dumps(doc, indent=2, sort_keys=True).split('\n')
    if not max_lines or len(lines) <= max_lines:
        print('\n'.join(lines))
    else:
        print('\n'.join(lines[:max_lines/2]) + '\n...\n' + '\n'.join(lines[-max_lines/2:]))

In [86]:
# This is the start of the query I need for full-text search
query_text = 'db-camp'
r = es.search(index='cbit', doc_type='sample', body={
        "query": {
            "match_phrase": {
                "_all": query_text
            }
        }
    })
p(r)

{
  "_shards": {
    "failed": 0, 
    "successful": 5, 
    "total": 5
  }, 
  "hits": {
    "hits": [
      {
        "_id": "5", 
        "_index": "cbit", 
        "_score": 0.36076978, 
        "_source": {
          "*Compound": "db-cAMP - dibutyryl-cAMP", 
          "*Material": "PS - polystyrene", 
...
        }, 
        "_type": "sample"
      }
    ], 
    "max_score": 0.36076978, 
    "total": 6
  }, 
  "timed_out": false, 
  "took": 4
}


In [87]:
# Really, I need a study ID in each sample,
# and I need to get the study IDs of all the samples that matched
import reader, config, json
cfg = config.Config()
a = reader.read_assay(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/a_transcription_micro_1.txt', 'r'))
s = reader.read_study_sample(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/s_study_sample.txt', 'r'))
d = reader.join_study_sample_and_assay(reader.clean_up_study_samples(s), reader.clean_up_assay(a))
d = reader.apply_special_treatments_to_study_sample(d)

result = []

for i, (k, v) in enumerate(d.iteritems()):
    vv = v.copy()
    vv['Sample Name'] = k
    vv['*Study ID'] = 1
    result.append({'id': i, 'studyId': 1, '_source': vv})

for r in result:
    print(r['id'])
    es.index(index='cbit', doc_type='sample', id=r['id'], body=r['_source'], refresh=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [90]:
# This is the start of the query I need for full-text search
query_text = 'db-camp'
r = es.search(index='cbit', doc_type='sample', body={
        "query": {
            "match_phrase": {
                "_all": query_text
            }
        },
        "aggs": {
            "matching_studies": {
                "terms": {
                    "field": "*Study ID"
                }
            }
        }
    })
r

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'matching_studies': {u'buckets': [{u'doc_count': 6,
     u'key': 1}],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 0}},
 u'hits': {u'hits': [{u'_id': u'5',
    u'_index': u'cbit',
    u'_score': 0.31567356,
    u'_source': {u'*Compound': u'db-cAMP - dibutyryl-cAMP',
     u'*Material': u'PS - polystyrene',
     u'*Strain': u'hMSC - human mesenchymal stromal cells',
     u'*Study ID': 1,
     u'Array or chip design': u'Illumina HT12v3',
     u'Assay Type': u'in vitro',
     u'Attach Duration (hours)': 12.5,
     u'Barcode': u'5342595019_B',
     u'Biological Replicate': 1,
     u'Biologically degradable': u'no',
     u'CAS number': u'362-74-3',
     u'Cell type': u'stromal cell',
     u'Compound': u'dibutyryl-cAMP',
     u'Compound abbreviation': u'db-cAMP',
     u'Control': False,
     u'Culture medium': u'alpha-MEM',
     u'Dose (mM)': 1.0,
     u'Dose Duration (hours)': 6.0,
     u'Gene

In [92]:
# Add "*Study ID" to studies as well
import reader, config, json
cfg = config.Config()

i = reader.read_investigation(cfg, open('../../data/new_ISAcreatorArchives/StudyID_01_archive/i_Investigation.txt', 'r'))
result = {
    'id': 1,
    'sampleIds': [k for k in range(0, 17+1)],
    '_source': reader.conform_investigation_to_schema(
                reader.remove_isa_name_prefixes(
                  reader.remove_empty_values_in_dict(
                    reader.flatten_investigation(
                      i
                    )
                  )
                )
              )
}
d = result['_source'].copy()
d['*Study ID'] = 1
es.index(index='cbit', doc_type='study', id=1, body=d, refresh=True)

{u'_id': u'1',
 u'_index': u'cbit',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'study',
 u'_version': 2,
 u'created': False}

In [93]:
# Now match studies or samples
# (now I get an aggregation result that gives me matching study IDs, but the count is of both
#  studies *and* samples that match)
query_text = 'db-camp'
r = es.search(index='cbit', doc_type=None, body={
        "query": {
            "match_phrase": {
                "_all": query_text
            }
        },
        "aggs": {
            "matching_studies": {
                "terms": {
                    "field": "*Study ID"
                }
            }
        }
    })
r

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'matching_studies': {u'buckets': [{u'doc_count': 7,
     u'key': 1}],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 0}},
 u'hits': {u'hits': [{u'_id': u'5',
    u'_index': u'cbit',
    u'_score': 0.31567356,
    u'_source': {u'*Compound': u'db-cAMP - dibutyryl-cAMP',
     u'*Material': u'PS - polystyrene',
     u'*Strain': u'hMSC - human mesenchymal stromal cells',
     u'*Study ID': 1,
     u'Array or chip design': u'Illumina HT12v3',
     u'Assay Type': u'in vitro',
     u'Attach Duration (hours)': 12.5,
     u'Barcode': u'5342595019_B',
     u'Biological Replicate': 1,
     u'Biologically degradable': u'no',
     u'CAS number': u'362-74-3',
     u'Cell type': u'stromal cell',
     u'Compound': u'dibutyryl-cAMP',
     u'Compound abbreviation': u'db-cAMP',
     u'Control': False,
     u'Culture medium': u'alpha-MEM',
     u'Dose (mM)': 1.0,
     u'Dose Duration (hours)': 6.0,
     u'Gene

---

OK, the queries I actually need are tricky, back to `ElasticSearch: The Definitive Guide`

---

In [96]:
p(es.cluster.health())

{
  "active_primary_shards": 10, 
  "active_shards": 10, 
  "active_shards_percent_as_number": 50.0, 
  "cluster_name": "elasticsearch", 
  "delayed_unassigned_shards": 0, 
  "initializing_shards": 0, 
  "number_of_data_nodes": 1, 
  "number_of_in_flight_fetch": 0, 
  "number_of_nodes": 1, 
  "number_of_pending_tasks": 0, 
  "relocating_shards": 0, 
  "status": "yellow", 
  "task_max_waiting_in_queue_millis": 0, 
  "timed_out": false, 
  "unassigned_shards": 10
}


In [105]:
es.indices.create(index='blogs', body={
        "settings": {
            "number_of_shards": 3,
            "number_of_replicas": 1
        }
    })

{u'acknowledged': True}

In [106]:
es.indices.get(index='blogs')

{u'blogs': {u'aliases': {},
  u'mappings': {},
  u'settings': {u'index': {u'creation_date': u'1478273698129',
    u'number_of_replicas': u'1',
    u'number_of_shards': u'3',
    u'uuid': u'JTNn6NNpROuaSNx3-AkUWw',
    u'version': {u'created': u'2040199'}}},
  u'warmers': {}}}

In [107]:
p(es.cluster.health())

{
  "active_primary_shards": 13, 
  "active_shards": 13, 
  "active_shards_percent_as_number": 50.0, 
  "cluster_name": "elasticsearch", 
  "delayed_unassigned_shards": 0, 
  "initializing_shards": 0, 
  "number_of_data_nodes": 1, 
  "number_of_in_flight_fetch": 0, 
  "number_of_nodes": 1, 
  "number_of_pending_tasks": 0, 
  "relocating_shards": 0, 
  "status": "yellow", 
  "task_max_waiting_in_queue_millis": 0, 
  "timed_out": false, 
  "unassigned_shards": 13
}


In [109]:
es.indices.put_settings(index='blogs', body={
        "number_of_replicas": 2
    })

{u'acknowledged': True}

In [110]:
es.indices.get(index='blogs')

{u'blogs': {u'aliases': {},
  u'mappings': {},
  u'settings': {u'index': {u'creation_date': u'1478273698129',
    u'number_of_replicas': u'2',
    u'number_of_shards': u'3',
    u'uuid': u'JTNn6NNpROuaSNx3-AkUWw',
    u'version': {u'created': u'2040199'}}},
  u'warmers': {}}}

In reality, it looks like I should store studies and samples in different *indices* in order to have different mappings for them (they're very different).  Maybe I can use for each sample the study id as the type: all samples look very similar, but samples from a given study look even more similar to each other.  Moreover, then I get the Study ID of the sample for free in ES' metadata

Interesting tidbit: I can let ES generate `id`s for me :-D  But they're shortGUIDs...

In [111]:
es.index(index='website', doc_type='blog', id=123, body={
        "title": "My first blog entry",
        "text":  "Just trying this out...",
        "date":  "2014/01/01"
    })

{u'_id': u'123',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 1,
 u'created': True}

In [112]:
es.index(index='website', doc_type='blog', body={
  "title": "My second blog entry",
  "text":  "Still trying this out...",
  "date":  "2014/01/01"
})

{u'_id': u'AVgwC0SFopLqMxzUh2_t',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 1,
 u'created': True}

In [114]:
p(es.get(index='website', doc_type='blog', id='123'))

{
  "_id": "123", 
  "_index": "website", 
  "_source": {
    "date": "2014/01/01", 
    "text": "Just trying this out...", 
    "title": "My first blog entry"
  }, 
  "_type": "blog", 
  "_version": 1, 
  "found": true
}


In [116]:
es.get(index='website', doc_type='blog', id='123', _source_include=['title', 'text'])

{u'_id': u'123',
 u'_index': u'website',
 u'_source': {u'text': u'Just trying this out...',
  u'title': u'My first blog entry'},
 u'_type': u'blog',
 u'_version': 1,
 u'found': True}

In [119]:
es.get_source(index='website', doc_type='blog', id='123')

{u'date': u'2014/01/01',
 u'text': u'Just trying this out...',
 u'title': u'My first blog entry'}

In [120]:
es.index(index='website', doc_type='blog', id='123', body={
        "title": "My first blog entry",
  "text":  "I am starting to get the hang of this...",
  "date":  "2014/01/02"
    })

{u'_id': u'123',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 2,
 u'created': False}

In [122]:
es.create(index='website', doc_type='blog', id='123', body={
        "title": "My first blog entry",
  "text":  "I am starting to get the hang of this...",
  "date":  "2014/01/02"
    })

ConflictError: TransportError(409, u'document_already_exists_exception', u'[blog][123]: document already exists')

In [123]:
es.index(index='website', doc_type='blog', id='123', op_type='create', body={
        "title": "My first blog entry",
  "text":  "I am starting to get the hang of this...",
  "date":  "2014/01/02"
    })

ConflictError: TransportError(409, u'document_already_exists_exception', u'[blog][123]: document already exists')

In [124]:
es.delete(index='website', doc_type='blog', id='123')

{u'_id': u'123',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 3,
 u'found': True}

In [125]:
es.delete(index='website', doc_type='blog', id='123')

NotFoundError: TransportError(404, u'{"found":false,"_index":"website","_type":"blog","_id":"123","_version":4,"_shards":{"total":2,"successful":1,"failed":0}}')

In [126]:
es.create(index='website', doc_type='blog', id='1', body={
        "title": "My first blog entry",
        "text":  "Just trying this out..."
    })

{u'_id': u'1',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 1,
 u'created': True}

In [127]:
es.get(index='website', doc_type='blog', id='1')

{u'_id': u'1',
 u'_index': u'website',
 u'_source': {u'text': u'Just trying this out...',
  u'title': u'My first blog entry'},
 u'_type': u'blog',
 u'_version': 1,
 u'found': True}

In [128]:
es.index(index='website', doc_type='blog', id='1', version=1, body={
  "title": "My first blog entry",
  "text":  "Starting to get the hang of this..."
})

{u'_id': u'1',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 2,
 u'created': False}

In [129]:
es.index(index='website', doc_type='blog', id='1', version=1, body={
  "title": "My first blog entry",
  "text":  "Starting to get the hang of this..."
})

ConflictError: TransportError(409, u'version_conflict_engine_exception', u'[blog][1]: version conflict, current [2], provided [1]')

In [130]:
es.create(index='website', doc_type='blog', id=2, version=5, version_type='external', body={
        "title": "My first external blog entry",
        "text":  "Starting to get the hang of this..."
    })

{u'_id': u'2',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 5,
 u'created': True}

In [132]:
es.index(index='website', doc_type='blog', id=2, version=10, version_type='external', body={
        "title": "My first external blog entry",
        "text":  "This is a piece of cake..."
    })

{u'_id': u'2',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 10,
 u'created': False}

In [133]:
es.update(index='website', doc_type='blog', id=1, body={
        "doc": {
            "tags": ["testing"],
            "views": 0
        }
    })

{u'_id': u'1',
 u'_index': u'website',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'blog',
 u'_version': 3}

In [134]:
es.get(index='website', doc_type='blog', id=1)

{u'_id': u'1',
 u'_index': u'website',
 u'_source': {u'tags': [u'testing'],
  u'text': u'Starting to get the hang of this...',
  u'title': u'My first blog entry',
  u'views': 0},
 u'_type': u'blog',
 u'_version': 3,
 u'found': True}

In [139]:
p(es.mget(body={
        "docs": [
            {
                "_index": "website",
                "_type": "blog",
                "_id": "2"
            },
            {
                "_index": "website",
                "_type": "blog",
                "_id": "1"
            }
        ]
    }))

{
  "docs": [
    {
      "_id": "2", 
      "_index": "website", 
      "_source": {
        "text": "This is a piece of cake...", 
        "title": "My first external blog entry"
      }, 
      "_type": "blog", 
      "_version": 10, 
      "found": true
    }, 
    {
      "_id": "1", 
...
        "text": "Starting to get the hang of this...", 
        "title": "My first blog entry", 
        "views": 0
      }, 
      "_type": "blog", 
      "_version": 3, 
      "found": true
    }
  ]
}


In [141]:
p(es.mget(index='website', doc_type='blog', body={
        "docs": [
            { "_id": "1" },
            { "_type": "pageviews", "_id": 1 }
        ]
    }))

{
  "docs": [
    {
      "_id": "1", 
      "_index": "website", 
      "_source": {
        "tags": [
          "testing"
        ], 
        "text": "Starting to get the hang of this...", 
        "title": "My first blog entry", 
        "views": 0
      }, 
      "_type": "blog", 
      "_version": 3, 
      "found": true
    }, 
    {
      "_id": "1", 
      "_index": "website", 
      "_type": "pageviews", 
      "found": false
    }
  ]
}


In [144]:
p(es.mget(index='website', doc_type='blog', body={
        "ids": ['1', '2']
    }))

{
  "docs": [
    {
      "_id": "1", 
      "_index": "website", 
      "_source": {
        "tags": [
          "testing"
        ], 
        "text": "Starting to get the hang of this...", 
        "title": "My first blog entry", 
        "views": 0
      }, 
      "_type": "blog", 
      "_version": 3, 
...
      "_source": {
        "text": "This is a piece of cake...", 
        "title": "My first external blog entry"
      }, 
      "_type": "blog", 
      "_version": 10, 
      "found": true
    }
  ]
}


In [146]:
import elasticsearch.helpers

In [150]:
elasticsearch.helpers.streaming_bulk??

In [151]:
list(elasticsearch.helpers.streaming_bulk(client=es, actions=[
        { "_op_type": "delete", "_index": "website", "_type": "blog", "_id": "123" }, 
        { "_op_type": "create", "_index": "website", "_type": "blog", "_id": "123",
          "_source": { "title":    "My first blog post" } },
        { "_op_type": "index", "_index": "website", "_type": "blog" ,
          "_source": { "title":    "My second blog post" } },
        { "_op_type": "update", "_index": "website", "_type": "blog", "_id": "123", "_retry_on_conflict" : 3,
          "_source": { "doc" : {"title" : "My updated blog post"} } }
    ]))

[(True,
  {u'delete': {u'_id': u'123',
    u'_index': u'website',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'blog',
    u'_version': 3,
    u'found': True,
    u'status': 200}}),
 (True,
  {u'create': {u'_id': u'123',
    u'_index': u'website',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'blog',
    u'_version': 4,
    u'status': 201}}),
 (True,
  {u'create': {u'_id': u'AVgxAv6mopLqMxzUh2_5',
    u'_index': u'website',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'blog',
    u'_version': 1,
    u'status': 201}}),
 (True,
  {u'update': {u'_id': u'123',
    u'_index': u'website',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'blog',
    u'_version': 5,
    u'status': 200}})]

In [152]:
list(elasticsearch.helpers.streaming_bulk(client=es, index='website', doc_type='blog', actions=[
            { "_op_type": "index",
              "_source": { "event": "User logged in" } },
            { "_op_type": "index", "_type": "blog",
              "_source": { "title": "Overriding the default type" } }
        ]))

[(True,
  {u'create': {u'_id': u'AVgxBdkxopLqMxzUh2_6',
    u'_index': u'website',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'blog',
    u'_version': 1,
    u'status': 201}}),
 (True,
  {u'create': {u'_id': u'AVgxBdkxopLqMxzUh2_7',
    u'_index': u'website',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'blog',
    u'_version': 1,
    u'status': 201}})]

In [167]:
es.indices.delete('cbit')

{u'acknowledged': True}

In [169]:
es.indices.delete('website')

{u'acknowledged': True}

In [172]:
es.indices.delete('megacorp')

{u'acknowledged': True}

In [173]:
es.search()

{u'_shards': {u'failed': 0, u'successful': 13, u'total': 13},
 u'hits': {u'hits': [{u'_id': u'14',
    u'_index': u'us',
    u'_score': 1.0,
    u'_source': {u'date': u'2014-09-24',
     u'name': u'John Smith',
     u'tweet': u'How many more cheesy tweets do I have to write?',
     u'user_id': 1},
    u'_type': u'tweet'},
   {u'_id': u'5',
    u'_index': u'gb',
    u'_score': 1.0,
    u'_source': {u'date': u'2014-09-15',
     u'name': u'Mary Jones',
     u'tweet': u'However did I manage before Elasticsearch?',
     u'user_id': 2},
    u'_type': u'tweet'},
   {u'_id': u'9',
    u'_index': u'gb',
    u'_score': 1.0,
    u'_source': {u'date': u'2014-09-19',
     u'name': u'Mary Jones',
     u'tweet': u'Geo-location aggregations are really cool',
     u'user_id': 2},
    u'_type': u'tweet'},
   {u'_id': u'8',
    u'_index': u'us',
    u'_score': 1.0,
    u'_source': {u'date': u'2014-09-18',
     u'name': u'John Smith',
     u'user_id': 1},
    u'_type': u'tweet'},
   {u'_id': u'10',
    u'

In [8]:
p(es.search(), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "14", 
        "_index": "us", 
        "_score": 1.0, 
        "_source": {
          "date": "2014-09-24", 
          "name": "John Smith", 
          "tweet": "How many more cheesy tweets do I have to write?", 
          "user_id": 1
        }, 
        "_type": "tweet"
      }, 
      {
        "_id": "5", 
        "_index": "gb", 
        "_score": 1.0, 
        "_source": {
          "date": "2014-09-15", 
          "name": "Mary Jones", 
          "tweet": "However did I manage before Elasticsearch?", 
          "user_id": 2
        }, 
        "_type": "tweet"
      }, 
      {
        "_id": "9", 
        "_index": "gb", 
        "_score": 1.0, 
        "_source": {
          "date": "2014-09-19", 
          "name": "Mary Jones", 
          "tweet": "Geo-location aggregations are really cool", 
          "user_id": 2
        }, 
        "_type": "twe

In [11]:
p(es.search(index='us,gb'))  # Search over 2+ indices

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 10
  }, 
  "hits": {
    "hits": [
      {
        "_id": "14", 
        "_index": "us", 
        "_score": 1.0, 
...
          "name": "Mary Jones", 
          "tweet": "The Query DSL is really powerful and flexible", 
          "user_id": 2
        }, 
        "_type": "tweet"
      }
    ], 
    "max_score": 1.0, 
    "total": 14
  }, 
  "timed_out": false, 
  "took": 4
}


In [14]:
p(es.search(index='us,gb', doc_type='tweet'))

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 10
  }, 
  "hits": {
    "hits": [
      {
        "_id": "14", 
        "_index": "us", 
        "_score": 1.0, 
...
          "name": "Mary Jones", 
          "tweet": "So yes, I am an Elasticsearch fanboy", 
          "user_id": 2
        }, 
        "_type": "tweet"
      }
    ], 
    "max_score": 1.0, 
    "total": 12
  }, 
  "timed_out": false, 
  "took": 4
}


In [16]:
p(es.search(doc_type='user'))

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "2", 
        "_index": "gb", 
        "_score": 1.0, 
...
          "email": "john@smith.com", 
          "name": "John Smith", 
          "username": "@john"
        }, 
        "_type": "user"
      }
    ], 
    "max_score": 1.0, 
    "total": 2
  }, 
  "timed_out": false, 
  "took": 3
}


In [21]:
len(es.search(body={"from": 10, "size": 5})["hits"]["hits"])

4

In [22]:
p(es.search(doc_type='tweet', q="tweet:elasticsearch"))

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "5", 
        "_index": "gb", 
        "_score": 0.375, 
...
          "name": "Mary Jones", 
          "tweet": "Elasticsearch is built for the cloud, easy to scale", 
          "user_id": 2
        }, 
        "_type": "tweet"
      }
    ], 
    "max_score": 0.375, 
    "total": 7
  }, 
  "timed_out": false, 
  "took": 46
}


In [23]:
p(es.search(doc_type='tweet', q="+name:john +tweet:mary"))

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "4", 
        "_index": "us", 
        "_score": 0.4585055, 
...
          "name": "John Smith", 
          "tweet": "@mary it is not just text, it does everything", 
          "user_id": 1
        }, 
        "_type": "tweet"
      }
    ], 
    "max_score": 0.4585055, 
    "total": 1
  }, 
  "timed_out": false, 
  "took": 7
}


In [25]:
#p(es.search(q='mary'))  #Same
p(es.search(q='_all:mary'))

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "4", 
        "_index": "us", 
        "_score": 0.25, 
...
          "name": "Mary Jones", 
          "tweet": "Elasticsearch is built for the cloud, easy to scale", 
          "user_id": 2
        }, 
        "_type": "tweet"
      }
    ], 
    "max_score": 0.25, 
    "total": 8
  }, 
  "timed_out": false, 
  "took": 7
}


In [26]:
p(es.search(q='+name:(mary john) +date:>2014-09-10 +(aggregations geo)'), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "9", 
        "_index": "gb", 
        "_score": 0.6457242, 
        "_source": {
          "date": "2014-09-19", 
          "name": "Mary Jones", 
          "tweet": "Geo-location aggregations are really cool", 
          "user_id": 2
        }, 
        "_type": "tweet"
      }
    ], 
    "max_score": 0.6457242, 
    "total": 1
  }, 
  "timed_out": false, 
  "took": 28
}


In [27]:
p(es.indices.get_mapping(index='gb', doc_type='tweet'))

{
  "gb": {
    "mappings": {
      "tweet": {
        "properties": {
          "date": {
            "format": "strict_date_optional_time||epoch_millis", 
            "type": "date"
          }, 
          "name": {
            "type": "string"
          }, 
          "tweet": {
            "type": "string"
          }, 
          "user_id": {
            "type": "long"
          }
        }
      }
    }
  }
}


In [33]:
p(es.indices.analyze(analyzer='spanish', body="Text to analyze"), None)

{
  "tokens": [
    {
      "end_offset": 4, 
      "position": 0, 
      "start_offset": 0, 
      "token": "text", 
      "type": "<ALPHANUM>"
    }, 
    {
      "end_offset": 7, 
      "position": 1, 
      "start_offset": 5, 
      "token": "to", 
      "type": "<ALPHANUM>"
    }, 
    {
      "end_offset": 15, 
      "position": 2, 
      "start_offset": 8, 
      "token": "analyz", 
      "type": "<ALPHANUM>"
    }
  ]
}


In [34]:
p(es.indices.analyze(analyzer='spanish', body="Texto por analizar"), None)

{
  "tokens": [
    {
      "end_offset": 5, 
      "position": 0, 
      "start_offset": 0, 
      "token": "text", 
      "type": "<ALPHANUM>"
    }, 
    {
      "end_offset": 18, 
      "position": 2, 
      "start_offset": 10, 
      "token": "analizar", 
      "type": "<ALPHANUM>"
    }
  ]
}


In [35]:
es.indices.delete("gb")

{u'acknowledged': True}

In [37]:
es.indices.create("gb", body={
        "mappings": {
            "tweet": {
                "properties": {
                    "tweet": {
                        "type": "string", 
                        "analyzer": "english"
                    },
                    "date": {
                        "type": "date"
                    },
                    "name": {
                        "type": "string"
                    },
                    "user_id": {
                        "type": "long"
                    }
                }
            }
        }
    })

{u'acknowledged': True}

In [39]:
es.indices.put_mapping(index='gb', doc_type='tweet', body={
        "properties": {
            "tag": {
                "type": "string",
                "index": "not_analyzed"
            }
        }
    })

{u'acknowledged': True}

In [40]:
p(es.indices.get_mapping(index='gb', doc_type='tweet'), None)

{
  "gb": {
    "mappings": {
      "tweet": {
        "properties": {
          "date": {
            "format": "strict_date_optional_time||epoch_millis", 
            "type": "date"
          }, 
          "name": {
            "type": "string"
          }, 
          "tag": {
            "index": "not_analyzed", 
            "type": "string"
          }, 
          "tweet": {
            "analyzer": "english", 
            "type": "string"
          }, 
          "user_id": {
            "type": "long"
          }
        }
      }
    }
  }
}


In [42]:
p(es.indices.analyze(index='gb', body={
        "field": "tweet",
        "text": "Black-cats"
    }))

{
  "tokens": [
    {
      "end_offset": 5, 
      "position": 0, 
      "start_offset": 0, 
      "token": "black", 
      "type": "<ALPHANUM>"
    }, 
    {
      "end_offset": 10, 
      "position": 1, 
      "start_offset": 6, 
      "token": "cat", 
      "type": "<ALPHANUM>"
    }
  ]
}


In [43]:
p(es.indices.analyze(index='gb', body={
        "field": "tag",
        "text": "Black-cats"
    }))

{
  "tokens": [
    {
      "end_offset": 10, 
      "position": 0, 
      "start_offset": 0, 
      "token": "Black-cats", 
      "type": "word"
    }
  ]
}


In [45]:
p(es.search(body={}))

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "14", 
        "_index": "us", 
        "_score": 1.0, 
...
          "email": "john@smith.com", 
          "name": "John Smith", 
          "username": "@john"
        }, 
        "_type": "user"
      }
    ], 
    "max_score": 1.0, 
    "total": 7
  }, 
  "timed_out": false, 
  "took": 3
}


In [46]:
p(es.search(body={
            "query": {
                "match_all": {}
            }
        }))

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "14", 
        "_index": "us", 
        "_score": 1.0, 
...
          "email": "john@smith.com", 
          "name": "John Smith", 
          "username": "@john"
        }, 
        "_type": "user"
      }
    ], 
    "max_score": 1.0, 
    "total": 7
  }, 
  "timed_out": false, 
  "took": 4
}


In [48]:
p(es.search(body={
            "query": {
                "match": {
                    "tweet": "elasticsearch"
                }
            }
        }), None)

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [
      {
        "_id": "10", 
        "_index": "us", 
        "_score": 0.3125, 
        "_source": {
          "date": "2014-09-20", 
          "name": "John Smith", 
          "tweet": "Elasticsearch surely is one of the hottest new NoSQL products", 
          "user_id": 1
        }, 
        "_type": "tweet"
      }, 
      {
        "_id": "6", 
        "_index": "us", 
        "_score": 0.3125, 
        "_source": {
          "date": "2014-09-16", 
          "name": "John Smith", 
          "tweet": "The Elasticsearch API is really easy to use", 
          "user_id": 1
        }, 
        "_type": "tweet"
      }, 
      {
        "_id": "12", 
        "_index": "us", 
        "_score": 0.25, 
        "_source": {
          "date": "2014-09-22", 
          "name": "John Smith", 
          "tweet": "Elasticsearch and I have left the honeymoon stage, and I still love her.", 
  

In [49]:
p(es.search(body={
            "query": {
                "bool": {
                    "must":     { "match": {"email": "business opportunity"} },
                    "must_not": { "match": {"name": "mary"} },
                    "should":   { "match": {"tweet": "full text"} },
                    "filter":   { "range": { "age": {"gt": 30}} }
                }
            }
        }))

{
  "_shards": {
    "failed": 0, 
    "successful": 10, 
    "total": 13
  }, 
  "hits": {
    "hits": [], 
    "max_score": null, 
    "total": 0
  }, 
  "timed_out": false, 
  "took": 23
}


In [52]:
p(es.indices.validate_query(index='us', doc_type='tweet', body={
        "query": {
            "tweet": {
                "match": "really powerful"
            }
        }
    }))

{
  "_shards": {
    "failed": 0, 
    "successful": 1, 
    "total": 1
  }, 
  "valid": false
}


In [53]:
p(es.indices.validate_query(index='us', doc_type='tweet', body={
        "query": {
            "match": {
                "tweet": "really powerful"
            }
        }
    }))

{
  "_shards": {
    "failed": 0, 
    "successful": 1, 
    "total": 1
  }, 
  "valid": true
}


In [55]:
p(es.indices.validate_query(body={
        "query": {
            "match": {
                "tweet": "really powerful"
            }
        }
    }, explain=True))

{
  "_shards": {
    "failed": 0, 
    "successful": 2, 
    "total": 3
  }, 
  "explanations": [
    {
      "explanation": "tweet:realli tweet:power", 
      "index": "gb", 
      "valid": true
    }, 
    {
      "explanation": "tweet:really tweet:powerful", 
      "index": "us", 
      "valid": true
    }
  ], 
  "valid": true
}
